In [ ]:
#Notebook для загрузки котировок акций с Yahoo Finance и MOEX 
#и запись в файл HDF5

from gz_import import *
from gz_mainlib import *
from gz_plotlib import *
from gz_dwnllib import *
from gz_rwlib import *
from gz_const import *

mainver()
plotver()
rwver()
dwnlver()
constver()

In [ ]:
reloadlibs()
from gz_import import *
from gz_mainlib import *
from gz_plotlib import *
from gz_dwnllib import *
from gz_rwlib import *
from gz_const import *

In [ ]:
#hdf5_contain('dat.h5')
#hdf5_writejson('dat.h5', 'yfinfo', yf_dict_new)
#hdf5_rewrite('dat.h5', 'datnew.h5')

In [2]:
#YAHOO

yahoo_for_load = us_tics_download + ['SXT', 'KIM', 'LSTR', 'STAG', 'DRE', 'EXPD'] 

#noloads = ['ALXN', 'LB', 'TOT']
#yahoo_for_load = list(set(yahoo_for_load) ^ set(noloads))

moex_for_load = ru_tics_download

print('Yahoo for load:', len(yahoo_for_load))

#загружаем
time1 = time.monotonic()
dfy, dfy_corr, dfy_doclh,nloaded_yahoo, err_yahoo = download_yahoo_mt(yahoo_for_load, pool_size=100)
time2 = time.monotonic()

#dfy1, dfy1_doclh = download_yf_tics(us_tics, t1, now) #reserve load

print('Load time, sec: {0:.2f}'.format(time2-time1))
print('Loaded yahoo:', nloaded_yahoo)
print('Error:', err_yahoo)
print('Yahoo loaded.')
print('')


if err_yahoo[0] != 'No errors' and err_yahoo[0] != 'No loads':
    #second attempt
    
    print('Another attempt..')
    dfy1, dfy1_corr, dfy1_doclh, nloaded_yahoo1, err_yahoo1 = \
                  download_yahoo_mt(err_yahoo, pool_size=100, load_index=False)

    if err_yahoo1[0] == 'No loads' or len(list(set(err_yahoo) ^ set(err_yahoo1))) == 0:
        print('No loads')
    else:
        print('Loads:', list(dfy1.columns[1:]))
        dfy = pd.merge(dfy, dfy1, on='date')
        dfy_doclh = pd.merge(dfy_doclh, dfy1_doclh, on='date')
        dfy_corr = correlation_returns(dfy)
        print('Loads err tics:', list(dfy1.coumns[1:]))

l_nodata_dfy = dfy.columns[dfy.isnull().any()].tolist()

if len(l_nodata_dfy) > 0:
    print('Yahoo NaN val:',l_nodata_dfy)
    dfy.drop(l_nodata_dfy, axis='columns', inplace=True)
    dfy_corr.drop(l_nodata_dfy, axis='columns', inplace=True)
    dfy_corr.drop(l_nodata_dfy, axis='index', inplace=True)
    print('Yahoo columns with NaN val deleted')

Yahoo for load: 348
Start load SP500, NASDAQ index... loaded
Load: 349. Errs(1): JCOM
Print thread stopped
Load time, sec: 11.14
Loaded yahoo: 349
Error: ['JCOM']
Yahoo loaded.

Another attempt..
Load: 0  . Errs(1): JCOM
Print thread stopped
No loads
No loads


In [5]:
dfy

,date,SP500,NASDAQ,ABBV,ABMD,AAPL,ADBE,ADSK,ACN,ABT,...,WFC,LSTR,YY,VLO,KIM,DRE,XRX,YUM,SXT,VZ
0,2020-01-02,3257.850098,9092.190430,80.137352,168.809998,73.894318,334.429993,187.830002,203.776093,84.001175,...,50.750061,108.867149,57.536419,83.632317,18.098192,32.768448,33.494518,98.588562,61.899940,55.290840
1,2020-01-03,3234.850098,9020.769531,79.376686,166.820007,73.175926,331.809998,184.949997,203.436722,82.977112,...,50.438477,108.780777,56.382164,80.458633,18.292101,33.369091,33.059177,98.279778,61.036533,54.702168
2,2020-01-06,3246.280029,9071.469727,80.003120,179.039993,73.759003,333.709991,187.119995,202.108261,83.411850,...,50.136337,108.080208,57.174492,80.467415,18.532177,33.006794,32.560341,98.221886,60.922672,54.584423
3,2020-01-07,3237.179932,9068.580078,79.546722,180.350006,73.412102,333.390015,187.500000,197.744751,82.948128,...,49.720890,108.521667,58.485249,81.522400,18.310570,32.797047,32.605682,98.395576,60.201584,53.977627
4,2020-01-08,3253.050049,9129.240234,80.110497,178.690002,74.593040,337.869995,189.949997,198.132614,83.286247,...,49.871964,109.644493,60.324223,81.838875,18.236698,32.987728,32.587543,98.569267,60.125675,54.077255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,2022-02-03,4477.439941,13878.820312,140.729996,290.519989,172.679993,510.829987,237.759995,345.549988,130.110001,...,55.169998,159.250000,47.220001,85.089996,24.040001,58.799999,21.940001,125.349998,82.160004,53.790001
528,2022-02-04,4500.529785,14098.009766,140.649994,295.649994,172.389999,513.539978,241.240005,347.500000,129.710007,...,56.330002,155.000000,48.160000,86.699997,23.830000,57.680000,22.040001,125.269997,81.379997,53.310001
529,2022-02-07,4483.870117,14015.669922,142.529999,289.839996,171.660004,507.100006,240.479996,343.959991,128.050003,...,57.330002,153.509995,46.070000,89.629997,23.799999,57.310001,21.700001,125.080002,82.239998,53.000000
530,2022-02-08,4521.540039,14194.450195,143.509995,309.029999,174.830002,511.309998,241.100006,345.070007,128.649994,...,58.750000,155.639999,47.099998,88.489998,24.120001,57.090000,21.389999,124.970001,82.980003,52.820000


In [3]:
#MOEX
print('MOEX for load:', len(moex_for_load))
#загружаем
time1 = time.monotonic()
dfm, dfm_corr, dfm_doclh, nloaded_moex, err_moex = download_moex_mt(moex_for_load)
time2 = time.monotonic()

print('Load time, sec: {0:.2f}'.format(time2-time1))
print('Loaded MOEX:', nloaded_moex)
print('Error:', err_moex)
print('MOEX loaded.')
print('')

l_nodata_dfm = dfm.columns[dfm.isnull().any()].tolist()


if len(l_nodata_dfm) > 0:
    print('MOEX NaN val:',l_nodata_dfm)
    dfm.drop(l_nodata_dfm, axis='columns', inplace=True)
    dfm_corr.drop(l_nodata_dfm, axis='columns', inplace=True)
    dfm_corr.drop(l_nodata_dfm, axis='index', inplace=True)
    print('MOEX columns with NaN val deleted')

MOEX for load: 121
Start load MOEX, RTSI index... loaded
Load: 122. Errs(1): UWGN_NaT_nodata
Print thread stopped
Load time, sec: 32.15
Loaded MOEX: 122
Error: ['UWGN_NaT_nodata']
MOEX loaded.

MOEX NaN val: ['SELGP']
MOEX columns with NaN val deleted


In [4]:
dfm

,date,IMOEX,RTSI,GTRK,SFIN,NSVZ,TCSG,PMSBP,KLSB,ISKJ,...,NVTK,AFKS,HYDR,VTBR,MAGN,TATN,SNGS,SNGSP,LKOH,SBER
0,2020-01-03,3076.37,1564.18,58.00,535.0,236.0,1332.4,94.2,11.74,13.22,...,1264.0,15.134,0.5547,0.046240,42.545,776.7,51.375,38.125,6294.0,255.00
1,2020-01-06,3078.87,1568.35,57.00,536.8,234.0,1322.8,94.6,11.73,13.08,...,1271.0,15.039,0.5603,0.046110,42.300,776.0,52.200,38.180,6382.0,253.90
2,2020-01-08,3110.06,1589.08,55.80,539.0,235.0,1350.2,97.0,11.75,13.03,...,1325.4,15.094,0.5649,0.046525,42.285,788.0,52.000,38.000,6496.5,259.15
3,2020-01-09,3118.08,1601.74,55.50,524.0,240.0,1366.8,95.8,11.77,13.20,...,1339.4,15.075,0.5829,0.047075,42.750,806.7,53.645,37.650,6430.0,257.99
4,2020-01-10,3123.66,1614.69,57.35,514.0,240.0,1398.4,95.8,11.87,13.10,...,1306.8,15.047,0.5882,0.047600,43.345,801.0,54.000,36.975,6490.0,258.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,2022-02-03,3471.18,1431.99,27.65,467.4,163.5,5235.0,133.2,10.31,91.20,...,1614.8,19.670,0.7252,0.041245,60.730,487.9,35.650,36.445,6865.5,250.79
529,2022-02-04,3471.01,1436.00,27.40,468.8,165.0,5184.5,134.0,10.27,92.56,...,1628.4,19.750,0.7398,0.041250,59.800,490.8,36.100,37.075,7010.0,256.53
530,2022-02-07,3470.78,1445.20,27.10,471.2,158.0,5005.0,133.0,10.18,91.58,...,1602.0,19.417,0.7443,0.041100,60.140,492.0,36.015,37.185,6988.0,257.61
531,2022-02-08,3551.65,1489.18,27.25,470.0,161.5,5067.5,132.4,10.20,92.30,...,1628.2,20.013,0.7551,0.042205,63.250,499.1,36.680,37.355,6957.5,269.80


In [ ]:
#aa,bb = download_yf_tics('BZ=F')

In [6]:
#df_brent, _, _, _, _  = download_yahoo_mt('BZ=F', date_start, date_end, load_index=False)
df_brent, _, _, _, _  = download_yahoo_mt('BZ=F', load_index=False)

i = 0    
while(df_brent is None and i < 5):
    #print('Brent (BZ=F) not loaded attempt=',i)
    df_brent, _, _, _, _  = download_yahoo_mt('BZ=F', load_index=False)
    i += 1
    sleep(5)
    
if(df_brent is None):
    print('Brent (BZ=F) not loaded!')
else:
    df_brent.rename(columns={'BZ=F': 'BRENT'}, inplace=True)
    print('BRENT loaded.')
    
    

df_brent

Load: 1  . Errs(0): 
Print thread stopped
BRENT loaded.


,date,BRENT
0,2020-01-02,66.250000
1,2020-01-03,68.599998
2,2020-01-06,68.910004
3,2020-01-07,68.269997
4,2020-01-08,65.440002
...,...,...
528,2022-02-03,91.110001
529,2022-02-04,93.269997
530,2022-02-07,92.690002
531,2022-02-08,90.779999


In [ ]:
#df_brent, _, _, _, _  = download_yahoo_mt('BZ=F', load_index=False)
#df_brent
#df_brent.drop(447, axis=0, inplace=True)
aaa, bbb = download_yf_tics('BZ=F')


In [7]:
#запись в HDF5
#df_instruments.to_hdf('dat.h5', 'df_instruments') #почему то увеличивает размер hdf5 при перезаписи

dfy.to_hdf('dat.h5', 'dfy')
dfy_corr.to_hdf('dat.h5', 'dfy_corr')
dfy_doclh.to_hdf('dat.h5', 'dfy_doclh')

dfm.to_hdf('dat.h5', 'dfm')
dfm_corr.to_hdf('dat.h5', 'dfm_corr')
dfm_doclh.to_hdf('dat.h5', 'dfm_doclh')

#df_brent.to_hdf('dat.h5', 'df_brent')

In [8]:
df_brent.to_hdf('dat.h5', 'df_brent')

In [9]:
hdf5_rewrite('dat.h5', 'dat2.h5')
#hdf5_delete_dataset('dat.h5', 'df_usindex')
#hdf5_contain('dat.h5')

['df_brent',
 'df_instruments',
 'dfm',
 'dfm_coint',
 'dfm_corr',
 'dfm_doclh',
 'dfy',
 'dfy_coint',
 'dfy_corr',
 'dfy_doclh',
 'l_industry_retrisksharpe',
 'sp500_tics',
 'us_tics_download',
 'us_tics_short',
 'yfinfo']

In [ ]:
#download_yahoo_doclh_2('BZ=F', date_start, date_now)

In [ ]:
#S&P 500
#payload = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#first_table = payload[0]
#second_table = payload[1]
#sp500_tics = len(sorted(first_table['Symbol'].tolist()))
#hdf5_writejson('dat.h5', 'sp500_tics', sp500_tics)
#hdf5_contain('dat.h5')
#sp500_tics = hdf5_readjson('dat.h5', 'sp500_tics')